# Tropical Homework 1
Data is saved as "TropHW1.txt"

## Import the modules to be used

In [13]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import pandas as pd
import metpy.calc as mpcalc
from metpy.plots import Hodograph, SkewT
import matplotlib.pyplot as plt
from metpy.units import units
from ipywidgets import widgets
from IPython.display import display
import IPython

# %matplotlib ipympl
%matplotlib widget

## Import the data

In [14]:
col_names = ['pressure', 'height', 'temperature', 'dewpoint', 'rel_hum', 'mixingratio', 'direction', 'speed']

df = pd.read_fwf('TropHW1.txt',
                 skiprows=1, usecols=[0, 1, 2, 3, 4, 5, 6, 7], names=col_names)

df['u_wind'], df['v_wind'] = mpcalc.wind_components(df['speed'],
                                                    np.deg2rad(df['direction']))

## Get the profile for all variables wanted in question 1.

In [15]:
p = df['pressure'].values * units.hPa
t = df['temperature'].values * units.degC
td = df['dewpoint'].values * units.degC
rh = df['rel_hum'] # (%)
w = df['mixingratio'] # (g/kg)
theta = mpcalc.potential_temperature(p, t)
thetaE = mpcalc.equivalent_potential_temperature(p, t, td)
thetaEs = mpcalc.saturation_equivalent_potential_temperature(p, t)

# don't need these, but it's fun to plot them
wind_speed = df['speed'].values * units.knots
wind_dir = df['direction'].values * units.degrees
u, v = mpcalc.wind_components(wind_speed, wind_dir)

## Interpolate between heights for question 1.
### A Few things to think about while doing the interpolation:
* x is always the value we want
* x must be increasing
* y is any of the other variables we have
### So, to fill this in:

| P (mb) | T (°C)  | Td (°C) | θ (K) | θe (K) | θes (K) | w (g/kg) | RH (%) |
|:------:|:-------:|:-------:|:-----:|:------:|:-------:|:--------:|:------:|
|   -    | 0       | -5      | 309   |   -    | -       | -        | -      |
| 1000   | 28      | 23      | -     |   -    | -       | -        | -      |
|   -    | -       | -       | 305   | 335    | 360     | -        | -      |
|   -    | -       | -       | 325   |   -    | -       | 6        | 80     |
| 750    | -       | 16      | -     |   -    | 370     | -        | -      |


### Two x arrays should cover all variables (Td and Theta)

In [24]:
# define conversion factors for hecto Pascals and Kelvin
hPa = 100.0
c_to_k = 273.15

# define the two y arrays
x_one = np.array(td) + c_to_k
x_two = np.array(theta)
x_three = np.array(t) + c_to_k

### Make all the others into arrays so we can interpolate to them

In [25]:
p_arr = np.array(p) * hPa
t_arr = np.array(t)
td_arr = np.array(td)
rh_arr = np.array(rh)
w_arr = np.array(w)
theta_arr = np.array(theta)
thetaE_arr = np.array(thetaE)
thetaEs_arr = np.array(thetaEs)

### Now get the variables we don't have

* Row one: Use Dew Point because it's already an array, but when looking at it by hand the dew point doesn't quite hit -5 when Temp is 0 which may throw off what he wants.

In [28]:
# Define the known value of our y to interpolate to
frz_td = c_to_k - 5.0

p_r1 = np.interp(-c_to_k, -x_three, p_arr)/hPa
thetaE_r1 = np.interp(-c_to_k, -x_three, thetaE_arr)
thetaEs_r1 = np.interp(-c_to_k, -x_three, thetaEs_arr)
w_r1 = np.interp(-c_to_k, -x_three, w_arr)
rh_r1 = np.interp(-c_to_k, -x_three, rh_arr)

print("Row one:")
print("P = " + str(p_r1))
print("ThetaE = " + str(thetaE_r1))
print("ThetaEs = " + str(thetaEs_r1))
print("w = " + str(w_r1))
print("rh = " + str(rh_r1))

Row one:
P = 578.4
ThetaE = 333.27346825682207
ThetaEs = 340.97140226205653
w = 4.212
rh = 64.2


* Row two: no need for interpretation because we have all the data we need.

In [19]:
p_r2 = 1000.0
print("Row two:")
print("Theta = " + str(theta[2]))
print("ThetaE = " + str(thetaE[2]))
print("ThetaEs = " + str(thetaEs[2]))
print("w = " + str(w[2]))
print("rh = " + str(rh[2]))

Row two:
Theta = 296.95 kelvin
ThetaE = 337.2476587028466 kelvin
ThetaEs = 351.76595291607714 kelvin
w = 13.94
rh = 74


* Row three: use Theta

In [20]:
# Define the known value of our y to interpolate to
theta305 = 305.0

p_r3 = np.interp(theta305, x_two, p_arr)/hPa
t_r3 = np.interp(theta305, x_two, t_arr)
td_r3 = np.interp(theta305, x_two, td_arr)
w_r3 = np.interp(theta305, x_two, w_arr)
rh_r3 = np.interp(theta305, x_two, rh_arr)

print("Row three:")
print("P = " + str(p_r3))
print("T = " + str(t_r3))
print("Td = " + str(td_r3))
print("w = " + str(w_r3))
print("rh = " + str(rh_r3))

Row three:
P = 814.3555356095063
T = 14.47085295843426
Td = 9.220363293756124
w = 9.76391118494885
rh = 77.03520964026183


* Row four: use Theta

In [21]:
# Define the known value of our y to interpolate to
theta325 = 325.0

p_r4 = np.interp(theta325, x_two, p_arr)/hPa
t_r4 = np.interp(theta325, x_two, t_arr)
td_r4 = np.interp(theta325, x_two, td_arr)
thetaE_r4 = np.interp(theta325, x_two, thetaE_arr)
thetaEs_r4 = np.interp(theta325, x_two, thetaEs_arr)

print("Row four:")
print("P = " + str(p_r4))
print("T = " + str(t_r4))
print("Td = " + str(td_r4))
print("ThetaE = " + str(thetaE_r4))
print("ThetaEs = " + str(thetaEs_r4))


Row four:
P = 504.48336029961536
T = -5.851663970038459
Td = -10.851663970038459
ThetaE = 336.3432722453775
ThetaEs = 341.48080914691207


* Row five: use dew point

In [22]:
# Define the known value of our y to interpolate to
td16 = c_to_k + 16.0

p_r5 = 750.0
t_r5 = np.interp(-td16, -x_one, t_arr)
theta_r5 = np.interp(-td16, -x_one, theta_arr)
thetaE_r5 = np.interp(-td16, -x_one, thetaE_arr)
w_r5 = np.interp(-td16, -x_one, w_arr)
rh_r5 = np.interp(-td16, -x_one, rh_arr)

print("Row five:")
print("T = " + str(t_r5))
print("Theta = " + str(theta_r5))
print("ThetaE = " + str(thetaE_r5))
print("w = " + str(t_r5))
print("Rh = " + str(rh_r5))

Row five:
T = 17.949999999999964
Theta = 299.2751476605689
ThetaE = 336.5698146212622
w = 17.949999999999964
Rh = 88.00000000000014


Plot it!
------------------

Radio buttons to turn things on and off

In [23]:
## most of this is stolen from here: https://unidata.github.io/MetPy/latest/tutorials/upperair_soundings.html

table_list = ['p_r1', 'p_r2', 'p_r3', 'p_r4', 'p_r5']

def PltSkew(show_table_level, rlim, llim, ppOn=False, BarbOn=True, hodOn=False):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.clear()
    skew = SkewT(fig, rotation=30)
    skew.plot(p, t, 'r')
    skew.plot(p, td, 'g')
    if BarbOn:
        skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1015, 400)
    skew.ax.set_xlim(llim, rlim)

    if ppOn:
        # Plot the parcel profile as a black line
        lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], t[0], td[0])
        parcel_prof = mpcalc.parcel_profile(p, t[0], td[0]).to('degC')
        skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
        skew.plot(p, parcel_prof, 'k', linewidth=2)
        skew.shade_cin(p, t, parcel_prof)
        skew.shade_cape(p, t, parcel_prof)

    # Plot a zero degree isotherm
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    
    if hodOn:
        ax_hod = inset_axes(skew.ax, '40%', '40%', loc=1)
        h = Hodograph(ax_hod, component_range=80.)
        h.add_grid(increment=20)
        h.plot_colormapped(u, v, wind_speed)
    
    skew.ax.axhline(eval(show_table_level), color='xkcd:baby shit brown', linestyle='--', linewidth=2)

    # Show the plot
    plt.show()

# This is because I want to add an option to plot the 
table_list = ['p_r1', 'p_r2', 'p_r3', 'p_r4', 'p_r5']
#show_table_level, rlim, llim, ppOn=False, BarbOn=True

show_table_level = widgets.Dropdown(options=table_list, value='p_r1', description='Table row to show', disabled=False)
rlim = widgets.IntSlider(min=20, max=60, value=30, description='Right Limit')
llim = widgets.IntSlider(min=-30, max=-10, value=-20, description='Left Limit')
ppOn = False
BarbOn = True
hodOn = False

widgets.interactive(PltSkew,
                    show_table_level=show_table_level,
                    rlim=rlim,
                    llim=llim,
                    ppOn=False,
                    BarbOn=True,
                    hodOn=False)

interactive(children=(Dropdown(description='Table row to show', options=('p_r1', 'p_r2', 'p_r3', 'p_r4', 'p_r5…

## Question 3
### Assess the stability of the following layers. For each layer, give a brief explanation in the form of temperatures or potential temperatures at the bottom and top of each layer. Additionally, out of the layers shown below, circle the one that is most unstable.

* 810–790 mb: 
* 950–900 mb:
* 475–400 mb: 
* 750–500 mb:
* Surface to 1000 mb:

In [12]:
%matplotlib widget
def SkewHl(highlight, annotOn=False):
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.clear()
    skew = SkewT(fig, rotation=30)
    skew.plot(p, t, 'r')
    skew.plot(p, td, 'g')
    skew.ax.set_ylim(1015, 400)
    skew.ax.set_xlim(-20, 30)

    # plot the highlight
    if hldict[highlight]:
        skew.ax.axhspan(hldict[highlight][0], hldict[highlight][1], alpha=.2)
    
    # annotate the layer
    props = dict(boxstyle="round", fc="0.8")
    if annotOn:
        if highlight != 'none':
            skew.ax.text(-15, hldict[highlight][1], highlight, ha="center", va="center", size=20,
                         bbox=props)

    # Plot a zero degree isotherm
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    
    # Show the plot
    plt.show()

hldict = {'none': False,
          '810-790': [810, 790],
          '950-900': [950, 900],
          '475-400': [470, 400],
          '750-500': [750, 500],
          'Sfc-1000': [p[0], 1000]}

highlight = widgets.Dropdown(options=list(hldict.keys()), value='none', description='Highlight layer', disabled=False)

widgets.interactive(SkewHl,
                    highlight=highlight,
                    annotOn=True)

interactive(children=(Dropdown(description='Highlight layer', options=('none', '810-790', '950-900', '475-400'…